# Gravity Drain Tank Modeling

In [1]:
from math import pi
import numpy as np
import branca
from ipywidgets import VBox, FloatSlider, Output, Button, Label
from ipycanvas import Canvas, MultiCanvas, hold_canvas

In [2]:
import numpy as np
from scipy.integrate import odeint
import time

In [3]:
class Plot(MultiCanvas):
    def __init__(self, x, y, color=None, scheme=branca.colormap.linear.RdBu_11):
        super(Plot, self).__init__(3, width=800, height=600, sync_image_data=True)

        self.color = color
        self.scheme = scheme
        
        self.background_color = '#f7f7f7'

        self.init_plot(x, y)

    def init_plot(self, x, y, color=None, scheme=None):
        self.x = x
        self.y = y
        self.color = color if color is not None else self.color
        self.scheme = scheme if scheme is not None else self.scheme

        padding = 0.0
        padding_x = padding * self.size[0]
        padding_y = padding * self.size[1]

        # TODO Fix drawarea max: It should be (canvas.size - padding)
        self.drawarea = (drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y) = (padding_x, padding_y, self.size[0] - 2 * padding_x, self.size[1] - 2 * padding_y)

        min_x, min_y, max_x, max_y = np.min(x), np.min(y), np.max(x), np.max(y)

        dx = max_x - min_x
        dy = max_y - min_y

        # Turns a data coordinate into pixel coordinate
        self.scale_x = lambda x: drawarea_max_x * (x - min_x) / dx + drawarea_min_x
        self.scale_y = lambda y: drawarea_max_y * (1 - (y - min_y) / dy) + drawarea_min_y

        # Turns a pixel coordinate into data coordinate
        self.unscale_x = lambda sx: (sx - drawarea_min_x) * dx / drawarea_max_x + min_x
        self.unscale_y = lambda sy: (1 - ((sy - drawarea_min_y) / drawarea_max_y)) * dy + min_y

        self.colormap = None
        if self.color is not None:
            self.colormap = self.scheme.scale(np.min(self.color), np.max(self.color))

    def draw_background(self):
        drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y = self.drawarea

        padding = 0.1
        offset_min_x = padding * self.size[0]
        offset_min_y = padding * self.size[1]
        offset_max_x = self.size[0] - 2 * offset_min_x 
        offset_max_y = self.size[1] - 2 * offset_min_y
        
        background = self[0]

        # Draw background
        background.fill_style = self.background_color
        background.global_alpha = 0.3
        background.fill_rect(drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y)
        background.global_alpha = 1

        # Draw grid and ticks
        n_lines = 10
        background.fill_style = 'black'
        background.stroke_style = '#8c8c8c'
        background.line_width = 1
        background.begin_path()

        for i in range(n_lines):
            j = i / (n_lines - 1)
            line_x = drawarea_max_x * j + drawarea_min_x
            line_y = drawarea_max_y * j + drawarea_min_y

            # Line on the y axis
            background.move_to(line_x, drawarea_min_y)
            background.line_to(line_x, drawarea_max_y + drawarea_min_y)

            # Line on the x axis
            background.move_to(drawarea_min_x, line_y)
            background.line_to(drawarea_max_x + drawarea_min_x, line_y)
            
            # Draw y tick
            background.text_align = 'right'
            background.text_baseline = 'middle'
            #background.fill_text('{0:.2e}'.format(self.unscale_y(line_y)), drawarea_min_x * 0.95, line_y)            
            background.fill_text('{0:.2f}'.format(self.unscale_y(line_y)), offset_min_x * 0.95, line_y)

            # Draw x tick
            background.text_align = 'center'
            background.text_baseline = 'top'
            #background.fill_text('{0:.2e}'.format(self.unscale_x(line_x)), line_x, drawarea_max_y + drawarea_min_y + drawarea_min_y * 0.05)
            background.fill_text('{0:.2f}'.format(self.unscale_x(line_x)), line_x, offset_max_y + offset_min_y + offset_min_y * 0.05)

        background.stroke()
        background.close_path()

In [4]:
class Torricelli_Path(Plot):
    def __init__(self, x, y, line_color='#3380FF', line_width=2, bkg=False, D=1, h0=1):
        super(Torricelli_Path, self).__init__(x, y)

        self.line_color = line_color
        self.line_width = line_width
        self.bkg = bkg
        self.D = D
        self.h0 = h0
        self.h = h0

        self.draw()

    def update(self, x, y, h, line_color=None, line_width=None):
        self.init_plot(x, y)
        self.h = h

        self.line_color = line_color if line_color is not None else self.line_color
        self.line_width = line_width if line_width is not None else self.line_width

        self.draw()
        
    def construct(self,D,h0):
        self.h0 = h0
        self.D = D

    def draw(self):
        with hold_canvas(self):
            self.clear()
            plot_layer = self[1]
            plot_layer.save()

            if self.bkg:
                self.draw_background()

            # Draw lines
            n_points = min(self.x.shape[0], self.y.shape[0])

            plot_layer.begin_path()
            plot_layer.stroke_style = self.line_color
            plot_layer.line_width = self.line_width
            plot_layer.line_join = 'bevel'
            plot_layer.line_cap = 'round'
            plot_layer.move_to(self.scale_x(self.x[1]), self.scale_y(self.y[1]))
            for idx in range(2, n_points):
                plot_layer.line_to(
                    self.scale_x(self.x[idx]), self.scale_y(self.y[idx])
                )

            plot_layer.stroke()
            plot_layer.close_path()
            plot_layer.fill_style = self.line_color
            plot_layer.fill_rect(0,self.scale_y(-self.h0),self.scale_x(self.D),self.scale_y(self.h))            
            plot_layer.stroke_style = "black"
            plot_layer.stroke_rect(0,self.scale_y(-self.h0),self.scale_x(self.D),self.scale_y(self.h0)) 
            plot_layer.restore()

# Experiment

![](tank.svg)

## Torricelli's law

Torricelli's law states the velocity of an incompressible liquid stream exiting a liquid tank at level $h$ below the surface is 

$$v = \sqrt{2gh}$$ 

This is the same velocity as an object dropped from a height $h$. The derivation is straightforward. From Bernoulli's principle,

$$\frac{v^2}{2} + gh + \frac{P}{\rho} = \text{constant}$$

Applying this principle, compare a drop of water just below the surface of the water at distance $h$ above the exit, to a drop of water exiting the tank

$$gh + \frac{P_{atm}}{\rho} = \frac{v^2}{2} + \frac{P_{atm}}{\rho}$$

$$\implies v^2 = 2gh$$
$$\implies v = \sqrt{2gh}$$

Torricelli's law is an approximation that doesn't account for the effects of fluid viscosity, the specific flow geometry near the exit, or other flow non-idealities. Nevertheless it is a useful first approximation for flow from a tank.

# Mass Balance for Tank with Constant Cross-Sectional Area

For a tank with constant cross-sectional area, such as a cylindrical or rectangular tank, the liquid height is described by a differential equation

$$A\frac{dh}{dt} = q_{in}(t) - q_{out}(t)$$

where $$q_{out}$$ is a function of liquid height. Torricelli's law tells the outlet flow from the tank is proportional to square root of the liquid height

$$ q_{out}(h) = C_v\sqrt{h} $$

Dividing by area we obtain a nonlinear ordinary differential equation 

$$ \frac{dh}{dt} = - \frac{C_V}{A}\sqrt{h} + \frac{1}{A}q_{in}(t) $$

in our standard form where the LHS derivative appears with a constant coefficient of 1.

# Equation Solving

## Construction parameters

In [5]:
A   = 1.0           # Tank area [meter^2]
h0 = 3              # tank height
Cv  = 0.1/60        # Outlet valve constant [cubic meters/sec/meter^1/2]

## Physical Constants

In [6]:
g = 9.81            # gravity constant

## Time Horizon

In [7]:
horizon = 20 * 60   # simulation period

## System Solving

In [8]:
# inlet flow rate in cubic meters/sec
def qin(t):
    return 0.0

def deriv(h,t):
    return qin(t)/A - Cv*np.sqrt(np.abs(h))/A

In [9]:
def solver():
    global D,t,h,v0
    D = 2 * np.sqrt(A/pi)
    IC = [h0]
    t = np.linspace(0,horizon,101)
    h = odeint(deriv,IC,t)
    v0 = np.sqrt(2*g*np.abs(h))

## Animating Gravity Drain Tank

In [10]:
solver()
slider = FloatSlider(description='replay time:', min=1, max=horizon, step=1)
button = Button(description="Run")
area = FloatSlider(description='area:', value=A, min=0.005, max=2, step=0.005)
initialheight = FloatSlider(description='initial height:', value=h0, min=0.1, max=3, step=0.1)
tapCv = FloatSlider(description='Cv:', value=Cv*60, min=0.01, max=0.2, step=0.01)

out = Output()
t1 = np.linspace(0,0.5,101)
x = v0[0][0] * t1 + D
y = -0.5 * g * t1 * t1 - h0
xmax = x[100]
ymax = y[100]
x = np.append(x,[xmax])
y = np.append(y,[ymax])
x = np.append([0.0],x)
y = np.append([0.0],y)

power_plot = Torricelli_Path(x, y, line_color='#3380FF', line_width=3, bkg=True, D=D, h0=h0)

def on_slider_change(change):
    i = int((slider.value/horizon) * 100)
    x = v0[i][0] * t1 + D
    y = -0.5 * g * t1 * t1 - h0
    x = np.append(x,[xmax])
    y = np.append(y,[ymax])
    x = np.append([0.0],x)
    y = np.append([0.0],y)
    
    if h[i][0] > 0:
        power_plot.update(x, y, h[i][0])
        with out:
            out.clear_output()
            print(f"t:{slider.value},h:{h[i][0]}")

slider.observe(on_slider_change, 'value')

def on_button_clicked(b):
    global A, h0, xmax, ymax, Cv
    A = area.value
    h0 = initialheight.value
    Cv = tapCv.value/60
    solver()
    x = v0[0][0] * t1 + D
    y = -0.5 * g * t1 * t1 - h0
    xmax = x[100]
    ymax = y[100]
    power_plot.construct(D,h0)
    with out:
        out.clear_output()
        print("Run Simulation.")
    for i in range(0,horizon + 1, 10):
        slider.value = i
        time.sleep(0.2)

button.on_click(on_button_clicked)

VBox((power_plot, slider, button, Label(value="$=====$ $Tank$ $Construction$ $Parameters$  $======$"), area, initialheight, tapCv, out))